In [ ]:
!pip install wandb

import os
from glob import glob
import numpy as np
import zipfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D, Dropout,Activation,MaxPooling2D,Flatten, BatchNormalization
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing import image
from keras.losses import categorical_crossentropy
from keras.callbacks import EarlyStopping
import pandas as pd
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
import random
from keras import regularizers
from keras.layers import MaxPooling2D, Input
from tensorflow.keras.losses import MeanSquaredError
from sklearn.model_selection import train_test_split
from PIL import Image
import wandb
from wandb.keras import WandbCallback

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.4 MB/s eta 0:00:00


In [ ]:
!wget http://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/celeba.zip


--2024-04-19 03:25:47--  http://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/celeba.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.194.120, 52.219.113.152, 52.219.193.96, ...
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.194.120|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1443490838 (1.3G) [application/zip]
Saving to: ‘celeba.zip’

celeba.zip          100%[===================>]   1.34G  89.3MB/s    in 16s     

2024-04-19 03:26:03 (85.0 MB/s) - ‘celeba.zip’ saved [1443490838/1443490838]



In [ ]:
with zipfile.ZipFile("celeba.zip","r") as zip_ref:
  zip_ref.extractall("data_faces/")

In [ ]:
path_cvs = "list.csv"
list = pd.read_csv(path_cvs)
list.head()

,image_id,Heavy_Makeup,High_Cheekbones,Male,Wavy_Hair,Young,Oval_Face,Pointy_Nose
0,000001.jpg,1,1,0,0,1,0,1
1,000002.jpg,0,1,0,0,1,0,0
2,000003.jpg,0,0,1,1,1,0,1
3,000004.jpg,0,0,0,0,1,0,1
4,000005.jpg,1,0,0,0,1,0,1


In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
list.replace(-1, 0, inplace=True)

# Guardar el DataFrame modificado de vuelta al archivo CSV
list.to_csv("data/list.csv", index=False)

In [ ]:
import tensorflow as tf

import pathlib
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
path_to_images = 'data_faces/img_align_celeba/'


df = pd.read_csv("list.csv")



files = tf.data.Dataset.from_tensor_slices(df.iloc[:150000, 0])
attributes = tf.data.Dataset.from_tensor_slices(df.iloc[:150000,1:].to_numpy())
data = tf.data.Dataset.zip((files, attributes))
print(data)


size_image = 80
def process_file(file_name, attributes):
    image = tf.io.read_file(path_to_images + file_name)
    image = tf.image.decode_jpeg(image, channels=1)
    image = tf.image.resize(image, [size_image, size_image])
    image /= 255.0  #
    return image, attributes

labeled_images = data.map(process_file,num_parallel_calls=tf.data.AUTOTUNE).batch(350).prefetch(tf.data.AUTOTUNE)

dataset = labeled_images
total_size = len(labeled_images)
print(len(labeled_images))
print(len(dataset))

train_size = int(0.8 * total_size)
validation_size = total_size - train_size


train = dataset.take(train_size)
val = dataset.skip(train_size)




<_ZipDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(7,), dtype=tf.int64, name=None))>
429
429


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
inputs = keras.Input(shape=(size_image, size_image, 1))
def block(x,base_filter=32, pooling=True):
    residual = x
    x = layers.Conv2D(base_filter, 3, activation="relu", padding="same", kernel_regularizer=regularizers.L1L2(l1=1e-4, l2=1e-4))(x)
    x = layers.Conv2D(2*base_filter, 3, activation="relu", padding="same", kernel_regularizer=regularizers.L1L2(l1=1e-4, l2=1e-4))(x)
    if (pooling == True):
        x = layers.MaxPooling2D(2, padding="same")(x)
    return x

x = block(inputs,base_filter=128)
x = block(x,base_filter=64)
x = block(x,base_filter=32)

for size in [32, 64]:
	residual = x
	x = layers.BatchNormalization()(x)
	#x = layers.Activation("relu")(x)
	x = layers.SeparableConv2D(size, 3, padding="same", use_bias=True, kernel_regularizer=regularizers.L1L2(l1=1e-4, l2=1e-4))(x)
	x = layers.BatchNormalization()(x)
	x = layers.Activation("relu")(x)
	x = layers.SeparableConv2D(size, 3, padding="same", use_bias=True,kernel_regularizer=regularizers.L1L2(l1=1e-4, l2=1e-4))(x)
	x = layers.MaxPooling2D(3, strides=2, padding="same")(x)
#	residual = layers.Conv2D(
#	size, 1, strides=2, padding="same", use_bias=False)(residual)
#	x = layers.add([x, residual])

#x = layers.GlobalAveragePooling2D()(x)
x = layers.Flatten()(x)
x= layers.Dense(100, activation="sigmoid", kernel_regularizer=regularizers.L1L2(l1=1e-4, l2=1e-4))(x)
x = layers.BatchNormalization()(x)
x= layers.Dense(50, activation="sigmoid", kernel_regularizer=regularizers.L1L2(l1=1e-4, l2=1e-4))(x)
x = layers.BatchNormalization()(x)
outputs = layers.Dense(7, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 80, 80, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 80, 80, 128)       1280      
                                                                 
 conv2d_1 (Conv2D)           (None, 80, 80, 256)       295168    
                                                                 
 max_pooling2d (MaxPooling2  (None, 40, 40, 256)       0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 40, 40, 64)        147520    
                                                                 
 conv2d_3 (Conv2D)           (None, 40, 40, 128)       73856     
                                                             

In [ ]:
wandb.init(project="ReconocimientoFacial",
          config={"Learning_rate":0.00005,
                 "epochs":20,
                 "batch_size":250,
                 "loss_function":"binary_crossentropy",
                 "optimizer":"RMSprop",
                  "ConvType":"Block, Sep"
                 })
config = wandb.config
run = wandb.run

early_stopping = EarlyStopping(monitor='val_binary_accuracy', patience=5, restore_best_weights=False)
checkpoint= ModelCheckpoint("FacialR.keras", save_best_only=True, monitor="val_binary_accuracy")
modellogger= WandbModelCheckpoint("models",save_best_only=True, monitor="val_binary_accuracy", mode="max",save_freq="epoch")
model.compile(loss="binary_crossentropy",optimizer=RMSprop(learning_rate=0.00005),metrics=['binary_accuracy'])
model.fit(train, epochs=config.epochs,verbose=1,validation_data=val,
          callbacks=[checkpoint,early_stopping,WandbMetricsLogger(),modellogger])
run.finish()

epoch/binary_accuracy,▁
epoch/epoch,▁
epoch/learning_rate,▁
epoch/loss,▁
epoch/val_binary_accuracy,▁
epoch/val_loss,▁
epoch/binary_accuracy,0.62737
epoch/epoch,0
epoch/learning_rate,5e-05
epoch/loss,2.3008
epoch/val_binary_accuracy,0.58606


Epoch 1/20
  6/343 [..............................] - ETA: 5:39 - loss: 2.1540 - binary_accuracy: 0.6676

343/343 [==============================] - ETA: 0s - loss: 2.0212 - binary_accuracy: 0.7068

wandb: Adding directory to artifact (./models)... Done. 0.0s


343/343 [==============================] - 397s 1s/step - loss: 2.0212 - binary_accuracy: 0.7068 - val_loss: 2.0239 - val_binary_accuracy: 0.6051
Epoch 2/20
343/343 [==============================] - ETA: 0s - loss: 1.8009 - binary_accuracy: 0.7508

...

wandb: Adding directory to artifact (./models)... Done. 0.0s


343/343 [==============================] - 388s 1s/step - loss: 0.8854 - binary_accuracy: 0.8408 - val_loss: 1.0172 - val_binary_accuracy: 0.7864
Epoch 13/20
343/343 [==============================] - ETA: 0s - loss: 0.8510 - binary_accuracy: 0.8423

wandb: Adding directory to artifact (./models)... Done. 0.0s


343/343 [==============================] - 387s 1s/step - loss: 0.8510 - binary_accuracy: 0.8423 - val_loss: 0.9851 - val_binary_accuracy: 0.7844


epoch/binary_accuracy,▁▃▅▆▆▇▇▇█████
epoch/epoch,▁▂▂▃▃▄▅▅▆▆▇▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▇▆▅▄▃▃▂▂▂▁▁▁
epoch/val_binary_accuracy,▁▅▆▇▆▇▇█▇▇█▇▇
epoch/val_loss,█▆▅▄▄▃▂▂▂▂▁▁▁
epoch/binary_accuracy,0.8423
epoch/epoch,12
epoch/learning_rate,5e-05
epoch/loss,0.85104
epoch/val_binary_accuracy,0.78442
